In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/inputdata/arabic_conversation.json
/kaggle/input/inputdata/arabic_conversation_simulation.json
/kaggle/input/inputdata/ChatData.py


In [2]:
!pip install transformers

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
# from ChatData import ChatData
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch
import torch.nn as nn


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
def train(path, model, optim,epochs):
    chatData = ChatData(path, tokenizer)
    chatData =  DataLoader(chatData, batch_size=120)#, shuffle=True)

    model.train()

    

    print("training .... ")

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            #loss = model(X, attention_mask=a, labels=X).loss
            losses = model(X, attention_mask=a, labels=X).loss
            loss = losses.mean()
            loss.backward()
            optim.step()
        # Save the model after each epoch
        torch.save(model.state_dict(), "model_state.pt")

        # Evaluation...
        model.eval()
        with torch.no_grad(): 
            print(infer("مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟"))
        model.train()
def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a,max_new_tokens=50 
                        ,pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id
                           )
    output = tokenizer.decode(output[0])
    return output


In [5]:
from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r", encoding="utf-8"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> "+i+" <bot>: "+self.X[idx+1]+" <endofstring>"
            except:
                break

        self.X = self.X[:5000]
        
        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [6]:
model_name = "aubmindlab/aragpt2-base"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
#device = "cpu" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])


model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))
optim = Adam(model.parameters(), lr=1e-4)



In [7]:
train("/kaggle/input/inputdata/arabic_conversation.json", model, optim,4)

<startofstring>  مرحبا عندي استفسار عن هذا المنتج <bot>:  مرحبا عنان كيف بنقدر نساعدك <endofstring>
training .... 


 25%|██▌       | 1/4 [00:11<00:34, 11.55s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>:,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


 50%|█████     | 2/4 [00:19<00:19,  9.54s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>:, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا بكم, مرحبا


 75%|███████▌  | 3/4 [00:27<00:08,  8.85s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>:, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج, مرحبا عن هذا المنتج


100%|██████████| 4/4 [00:35<00:00,  8.97s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>: مرحبا عندي استفسار عن هذا المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج, هل المنتج,


In [8]:
train("/kaggle/input/inputdata/arabic_conversation_simulation.json", model, optim,4)

<startofstring> مرحبًا <bot>: مرحبًا بك في مؤسسة اكسبو شوب! كيف يمكنني مساعدتك؟ <endofstring>
training .... 


 25%|██▌       | 1/4 [00:59<02:57, 59.15s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>: لدينا عرض لمكنسة كهربائية متاحة للبيع. سازودكم  ببعض الموديلات الشهيرة هي<pad> - شاشة 20 بوصة، معالج ثماني النواة، ذاكرة 16 غيغابايت، ذاكرة 16 غيغابايت، ذاكرة 16 غيغابايت، ذاكرة 16 غيغابايت�


 50%|█████     | 2/4 [02:03<02:04, 62.06s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>: لدينا عرض لمكنسة متاحة للبيع. سازودكم  ببعض الموديلات الشهيرة هي 
 - مكنسة كهربائية ABC: شاشة OLED، موصلات HDMI، موصلات HDMI، موصلات HDMI، موص


 75%|███████▌  | 3/4 [03:06<01:02, 62.77s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>: لدينا عرض لمكنسة متاحة للبيع. سازودكم  ببعض الموديلات الشهيرة هي 
 - مكنسة كهربائية ABC: قوة شفط عالية، سعة وعاء الغبار 2 لتر، فلتر 2 لتر، فلتر 4K، فلتر 4


100%|██████████| 4/4 [04:10<00:00, 62.51s/it]

<startofstring> مرحبًا! أنا أبحث عن منتجات جديدة. هل يمكنك مساعدتي؟ <bot>: بالطبع! أنا هنا لمساعدتك في العثور على المنتجات المناسبة. يرجى أن تخبرني بما تبحث عنه بالضبط أو اطرح عليّ أسئلة حول المنتجات المتاحة <bot>: أهلاً <endofstring>


In [9]:
print("infer from model : ")
while True:
  inp = input()
  if inp == "-1":
    break;
  print(infer(inp))


for i, batch in enumerate(chatData):
    for sequence in batch:
        input_tensor = sequence[0].tolist()  # Adjust this line according to your actual data structure
        print(tokenizer.decode(input_tensor))
    if i == 0:  # stop after first batch
        break

infer from model : 


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.